In [ ]:
import numpy as np
import keras
import os
from pathlib import Path
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = pd.read_csv('/content/drive/MyDrive/Data sets/ara.txt', sep='\t')


print(data_path.head(20))

       Hi.         مرحبًا.  \
0     Run!           اركض!   
1    Duck!      اخفض رأسك!   
2    Duck!     اخفضي رأسك!   
3    Duck!  اخفضوا رؤوسكم!   
4    Help!         النجدة!   
5    Jump!           اقفز!   
6    Stop!             قف!   
7    Stop!          توقف !   
8    Wait!           إنتظر   
9   Go on.           داوم.   
10  Go on.          استمر.   
11  Hello!         مرحباً.   
12  Hello.          أهلاً.   
13  Hello.         مرحباً.   
14  Hurry!          تعجّل!   
15  Hurry!         استعجل!   
16  I see.         انا اري   
17  I won!       أنا فُزت!   
18  Relax.          استرح.   
19  Smile.          ابتسم.   

   CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #629296 (Samer)  
0   CC-BY 2.0 (France) Attribution: tatoeba.org #9...                          
1   CC-BY 2.0 (France) Attribution: tatoeba.org #2...                          
2   CC-BY 2.0 (France) Attribution: tatoeba.org #2...                          
3   CC-BY 2.0 (France) Attribution: tatoeba.org #

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_chars = []
target_chars = []
with open('/content/drive/MyDrive/Data sets/ara.txt', "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines:
    if len(line.split("\t")) == 3: #Added this condition to only process lines with 3 values
        input_text, target_text, _ = line.split("\t")
        target_text = "\t" + target_text + "\n"
        # We use "tab" as the "start sequence" character
        input_texts.append(input_text)
        target_texts.append(target_text)
        for char in input_text:
            if char not in input_chars:
                input_chars.append(char)
        for char in target_text:
            if char not in target_chars:
                target_chars.append(char)
    elif len(line.strip()) != 0: #Added this to print lines that do not have 3 values but are not empty
        print(f"Line with less than 3 values: {line}")

In [ ]:
input_chars = sorted(input_chars)
target_chars = sorted(target_chars)

num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 12523
Number of unique input tokens: 74
Number of unique output tokens: 104
Max sequence length for inputs: 202
Max sequence length for outputs: 219


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])

In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '%': 3,
 "'": 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '3': 11,
 '4': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 ';': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'X': 44,
 'Y': 45,
 'a': 46,
 'b': 47,
 'c': 48,
 'd': 49,
 'e': 50,
 'f': 51,
 'g': 52,
 'h': 53,
 'i': 54,
 'j': 55,
 'k': 56,
 'l': 57,
 'm': 58,
 'n': 59,
 'o': 60,
 'p': 61,
 'q': 62,
 'r': 63,
 's': 64,
 't': 65,
 'u': 66,
 'v': 67,
 'w': 68,
 'x': 69,
 'y': 70,
 'z': 71,
 '’': 72,
 '€': 73}

In [ ]:
target_token_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '/': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'F': 22,
 'M': 23,
 'P': 24,
 'T': 25,
 '\\': 26,
 'a': 27,
 'b': 28,
 'e': 29,
 'f': 30,
 'g': 31,
 'i': 32,
 'l': 33,
 'o': 34,
 'r': 35,
 't': 36,
 'x': 37,
 '،': 38,
 '؟': 39,
 'ء': 40,
 'آ': 41,
 'أ': 42,
 'ؤ': 43,
 'إ': 44,
 'ئ': 45,
 'ا': 46,
 'ب': 47,
 'ة': 48,
 'ت': 49,
 'ث': 50,
 'ج': 51,
 'ح': 52,
 'خ': 53,
 'د': 54,
 'ذ': 55,
 'ر': 56,
 'ز': 57,
 'س': 58,
 'ش': 59,
 'ص': 60,
 'ض': 61,
 'ط': 62,
 'ظ': 63,
 'ع': 64,
 'غ': 65,
 'ـ': 66,
 'ف': 67,
 'ق': 68,
 'ك': 69,
 'ل': 70,
 'م': 71,
 'ن': 72,
 'ه': 73,
 'و': 74,
 'ى': 75,
 'ي': 76,
 'ً': 77,
 'ٌ': 78,
 'ٍ': 79,
 'َ': 80,
 'ُ': 81,
 'ِ': 82,
 'ّ': 83,
 'ْ': 84,
 '٠': 85,
 '١': 86,
 '٢': 87,
 '٣': 88,
 '٤': 89,
 '٥': 90,
 '٦': 91,
 '٧': 92,
 '٨': 93,
 '٩': 94,
 '٪': 95,
 'ٱ': 96,
 'ی': 97,
 '۰': 98,
 '۱': 99,
 '۹': 10

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype="float32")

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype="float32")

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype="float32")

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):

    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0

    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0

    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0

        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.GRU(500, return_state=True)
_ , state_h = encoder(encoder_inputs)

# Set up the decoder, using `state_h` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences.
decoder = keras.layers.GRU(500, return_sequences=True, return_state=True)

# Ensure the decoder receives the correct initial state (state_h)
decoder_outputs, _ = decoder(decoder_inputs, initial_state=state_h)

decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=20,
    validation_split=0.2)

#Save model
model.save("/content/drive/MyDrive/Data sets/s2s_model.keras")

Epoch 1/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 873s 6s/step - accuracy: 0.8880 - loss: 0.7441 - val_accuracy: 0.8583 - val_loss: 0.6445
Epoch 2/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 925s 6s/step - accuracy: 0.9206 - loss: 0.3203 - val_accuracy: 0.8602 - val_loss: 0.5536
Epoch 3/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 918s 6s/step - accuracy: 0.9259 - loss: 0.2838 - val_accuracy: 0.8643 - val_loss: 0.5140
Epoch 4/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 924s 6s/step - accuracy: 0.9304 - loss: 0.2650 - val_accuracy: 0.8736 - val_loss: 0.4877
Epoch 5/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 920s 6s/step - accuracy: 0.9326 - loss: 0.2586 - val_accuracy: 0.8747 - val_loss: 0.4757
Epoch 6/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 873s 6s/step - accuracy: 0.9346 - loss: 0.2481 - val_accuracy: 0.8794 - val_loss: 0.4610
Epoch 7/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 918s 6s/step - accuracy: 0.9354 - loss: 0.2476 - val_accuracy: 0.8785 - val_loss: 0.4637
Epoch 8/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 857s 5s/step - accuracy: 0.9363 - loss: 0.2398 - val_accu

In [ ]:
model = keras.models.load_model("/content/drive/MyDrive/Data sets/s2s_model.keras")

In [ ]:
#Reverse-Lookup Dictionaries

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


In [ ]:
target_seq = np.zeros((1, 1, num_decoder_tokens), dtype="float32")
target_seq[0, 0, target_token_index["\t"]] = 1.0
output_sentence = ""
sampled_char = ''

while len(output_sentence) < max_decoder_seq_length and sampled_char != "\n": # Use "and"
    output = model.predict([encoder_input_data[0:1], target_seq])
    sampled_token_index = np.argmax(output[-1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    output_sentence += sampled_char

    # Update the target sequence for the next iteration
    target_seq = np.zeros((1, 1, num_decoder_tokens), dtype="float32")
    target_seq[0, 0, sampled_token_index] = 1.0

print(output_sentence)
print(len(output_sentence))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/

In [ ]:
target_seq = np.zeros((1, 1, num_decoder_tokens), dtype="float32")
target_seq[0, 0, target_token_index[" "]] = 1.0

decoded_sentence = ""
stop_condition = False
while not stop_condition:
  output_tokens = model.predict([encoder_input_data[0:1], target_seq])
  sampled_token_index = np.argmax(output_tokens[0, -1, :])
  sampled_char = reverse_target_char_index[sampled_token_index]
  decoded_sentence += sampled_char

  if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
    stop_condition = True

  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, sampled_token_index] = 1.0

print(decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


In [ ]:
# Decoding Sequences
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: مذكر هذا؟

-
Input sentence: Run!
Decoded sentence: اقفض!

-
Input sentence: Duck!
Decoded sentence: اخفضر على كسي.

-
Input sentence: Duck!
Decoded sentence: اخفضر على كسي.

-
Input sentence: Duck!
Decoded sentence: اخفضر على كسي.

-
Input sentence: Help!
Decoded sentence: من خبر!

-
Input sentence: Jump!
Decoded sentence: اقفض!

-
Input sentence: Stop!
Decoded sentence: قال!

-
Input sentence: Stop!
Decoded sentence: قال!

-
Input sentence: Wait!
Decoded sentence: إنهُ مادي.

-
Input sentence: Go on.
Decoded sentence: استمر

-
Input sentence: Go on.
Decoded sentence: استمر

-
Input sentence: Hello!
Decoded sentence: ماذهب

-
Input sentence: Hello.
Decoded sentence: ماذهب ي.

-
Input sentence: Hello.
Decoded sentence: ماذهب ي.

-
Input sentence: Hurry!
Decoded sentence: ماذهب

-
Input sentence: Hurry!
Decoded sentence: ماذهب

-
Input sentence: I see.
Decoded sentence: أنا مشتاق.

-
Input sentence: I won!
Decoded sentence: أنا ذلك.

-
Input sente